In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2024-04-28 20:01:21.123200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 20:01:21.177630: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 20:01:21.177677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 20:01:21.177715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 20:01:21.187236: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 20:01:30.969396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [12]:
from models.models import TransformerModel 

model = TransformerModel(
    running_units=256,
    d=64,
    h=4,
    ffn_mult=1,
    depth=3,
    pos_type='learned', # learned
    prec_type="embed_input", # embed_input | pretoken | inject_pre | inject_ffn
    learned_pos=True,
    prenorm=False,
    norm_type="layer",
    penultimate_units=None,
    output_units=174,
    max_charge=6,
    sequence_length=30,
    alphabet=False,
    dropout=0,
    precursor_units=None,
    inject_position="all" # all | pre | post    
)


from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='adam', 
            loss=masked_spectral_distance,
            metrics=[masked_pearson_correlation_distance])

model.build(input_shape=(None, 30))

model.summary()
#models

ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'

In [2]:
# Prosit dataset

import os
os.environ['HF_HOME'] = "/cmnfs/proj/prosit_astral"
os.environ['HF_DATASETS_CACHE'] = "/cmnfs/proj/prosit_astral/datasets"

print("[UNIMOD:1]-K[UNIMOD:1]".count('[UNIMOD:' + '1' + ']'))

import numpy as np
from dlomix.data import FragmentIonIntensityDataset
import pandas as pd

from datasets import disable_caching
disable_caching()

PTMS_ALPHABET = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M[UNIMOD:35]": 21,
    "R[UNIMOD:7]":22,
    "C[UNIMOD:4]": 2,
    "Q[UNIMOD:7]":4,
    "N[UNIMOD:7]":3,
}

rt_data = FragmentIonIntensityDataset(
    data_source="/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_train.parquet",
    val_data_source="/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_val.parquet",
    test_data_source="/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_test.parquet",
    data_format="parquet", 
    val_ratio=0.2, max_seq_len=30, encoding_scheme="naive-mods",
    vocab=PTMS_ALPHABET,
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed","method_nbr"],
    batch_size=2048
)

2


/nfs/home/students/d.lochert/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: The parsed sequence
    - _n_term_mods: The N-terminal modifications
    - _c_term_mods: The C-terminal modifications



/nfs/home/students/d.lochert/projects/astral/dlomix/src/dlomix/data/dataset.py:169: UserWarning: 
                    Multiple data sources provided {'train': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_train.parquet', 'val': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_val.parquet', 'test': '/cmnfs/data/proteomics/Prosit_PTMs/Transformer_Train/clean_test.parquet'}, please ensure that the data sources are already split into train, val and test sets
                    since no splitting will happen. If not, please provide only one data source and set the val_ratio to split the data into train and val sets."
                    
  warnings.warn(
Filter (num_proc=2): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 814/814 [00:00<00:00, 4282.56 examples/s]


In [18]:
rt_data["train"]

Dataset({
    features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
    num_rows: 2836
})

In [14]:
rt_data.tensor_train_data

<_PrefetchDataset element_spec=({'modified_sequence': TensorSpec(shape=(None, 30), dtype=tf.float32, name=None), 'precursor_charge_onehot': TensorSpec(shape=(None, 6), dtype=tf.int64, name=None), 'collision_energy_aligned_normed': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'method_nbr': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 174), dtype=tf.float32, name=None))>

In [26]:
from models.models import TransformerModel 


model = TransformerModel(
    running_units=256,
    d=64,
    h=4,
    ffn_mult=1,
    depth=3,
    pos_type='learned', # learned
    prec_type="pretoken", # embed_input | pretoken | inject_pre | inject_ffn
    learned_pos=True,
    prenorm=False,
    norm_type="layer",
    penultimate_units=None,
    output_units=174,
    max_charge=6,
    sequence_length=30,
    alphabet=False,
    dropout=0,
    precursor_units=None,
    inject_position="all" # all | pre | post    
)


from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='adam', 
            loss=masked_spectral_distance,
            metrics=[masked_pearson_correlation_distance])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from callbacks import CyclicLR, LearningRateLogging

cyclicLR = CyclicLR(base_lr=0.000001, max_lr=0.0002, step_size=2, mode='triangular',
                 gamma=0.95)

early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=20,
    restore_best_weights=True)

learningRate = LearningRateLogging()


model.fit(
    rt_data.tensor_train_data,
    validation_data=rt_data.tensor_val_data,
    epochs=4,
#    callbacks=[
#        WandbCallback(save_model=False),
#        cyclicLR,
#        early_stopping,
        #save_best,
#        learningRate
#    ]
)

model.summary()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1/4
2/2 [==============================] - 16s 4s/step - loss: 0.7404 - masked_pearson_correlation_distance: 0.6821 - val_loss: 0.7023 - val_masked_pearson_correlation_distance: 0.6291
Epoch 2/4
2/2 [==============================] - 8s 3s/step - loss: 0.7113 - masked_pearson_correlation_distance: 0.6377 - val_loss: 0.6785 - val_masked_pearson_correlation_distance: 0.5939
Epoch 3/4
2/2 [==============================] - 8s 2s/step - loss: 0.6879 - masked_pearson_correlation_distance: 0.6046 - val_loss: 0.6720 - val_masked_pearson_correlation_distance: 0.5884
Epoch 4/4
2/2 [==============================] - 8s 3s/step - loss: 0.6688 - masked_pearson_correlation_distance: 0.5802 - val_loss: 0.6615 - val_masked_pearson_correlation_distance: 0.5764
Model: "transformer_model_15"
_________________________________________________________________
 Layer (type)                Output Shape              

In [ ]:
intdata[

In [22]:
ALPHABET_UNMOD = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M[UNIMOD:35]": 21,
    "R[UNIMOD:7]":22,
    "C[UNIMOD:4]": 2,
    "Q[UNIMOD:7]":4,
    "N[UNIMOD:7]":3,
}

from tensorflow.keras.layers.experimental import preprocessing
string_lookup = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))


input_embedding = tf.one_hot(string_lookup(["A", "S", "V"]), len(ALPHABET_UNMOD))

input_embedding

for i in rt_data.tensor_train_data:
    print(i)

train_data[0][0]

({'modified_sequence': <tf.Tensor: shape=(2048, 30), dtype=float32, numpy=
array([[nan,  3., 16., ...,  0.,  0.,  0.],
       [nan, 15.,  3., ...,  0.,  0.,  0.],
       [nan,  3., 17., ...,  0.,  0.,  0.],
       ...,
       [nan, 22., 16., ...,  0.,  0.,  0.],
       [nan, 18., 10., ...,  0.,  0.,  0.],
       [nan,  8., 17., ...,  0.,  0.,  0.]], dtype=float32)>, 'precursor_charge_onehot': <tf.Tensor: shape=(2048, 6), dtype=int64, numpy=
array([[0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])>, 'collision_energy_aligned_normed': <tf.Tensor: shape=(2048,), dtype=float32, numpy=
array([0.35      , 0.2860499 , 0.35      , ..., 0.35      , 0.28745976,
       0.28614625], dtype=float32)>, 'method_nbr': <tf.Tensor: shape=(2048,), dtype=int64, numpy=array([2, 1, 2, ..., 2, 1, 1])>}, <tf.Tensor: shape=(2048, 174), dtype=float32, numpy=
array([[ 0.        ,  0.        , -1. 

NameError: name 'train_data' is not defined

In [1]:
from copy import deepcopy
import numpy as np
import tensorflow as tf
K = tf.keras
L = K.layers
A = K.activations
I = K.initializers

2024-05-21 13:04:50.900545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 13:04:50.900601: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 13:04:50.900640: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



class AdaptiveNorm(L.Layer):

    def __init__(self):
        super(AdaptiveNorm, self).__init__()
        self.norm = L.LayerNormalization()
        self.norm.trainable = False

    



layer = AdaptiveNorm()
x = tf.random.normal((10,30,512))


layer(x)



In [3]:
layer.trainable_variables


[]

In [70]:
class AdaptiveNorm(L.Layer):

    def __init__(self):
        super(AdaptiveNorm, self).__init__()
        self.norm = L.LayerNormalization()
        self.norm.trainable = False
        
        
    def build(self, x):
        self.scalebias = L.Dense(2*x[-1], 
                                 use_bias=False, 
                                 kernel_initializer=I.RandomNormal(0, 0.1 * x[-1]**-0.5)) 
    
    def call(self, x, metadata):
         
        scalebias = self.scalebias(metadata)  # (bs, 2*ru)
        scale, bias = tf.split(scalebias, 2, -1) # 2 x (bs, ru)
        
        print(scale.numpy().mean())
        print(scale.numpy().std())
        
        scale = scale[:,None]  # (bs, 1, ru)
        bias = bias[:,None]    # (bs, 1, ru)
        
        out = self.norm(x)
        out = (1 + scale) * out - bias 
        
        return out
        
        
class BaseNorm(L.Layer):

    def __init__(self, norm):
        super(BaseNorm, self).__init__()
        self.norm = norm

    def __call__(self, x, metadata=None):
        if isinstance(self.norm, AdaptiveNorm):
            return self.norm(x, metadata)
        else:
            return self.norm(x)


    
layer = AdaptiveNorm()
x = tf.random.normal((10,30,256))




metadata = tf.random.normal((10,256))

layer(x, metadata)

layer.trainable_variables

base_layer = BaseNorm(layer)

out = base_layer(x, metadata)






-0.0013750743
0.10305293
-0.0013750743
0.10305293


In [5]:
x = tf.random.normal((10,512))

In [36]:
out.numpy().mean()

0.0047188704

In [7]:
tf.split(x, 2, -1)[0].shape

TensorShape([10, 256])